In [1]:
import pandas as pd
from io import StringIO
from sklearn.metrics import f1_score

# Read data from the CSV file and select only the required columns.
df = pd.read_csv('results_rs_source.csv')
df = df[['filename', 'toolid', 'findings']]

# A dictionary mapping each tool to the string(s) it produces for a reentrancy finding.
# You can easily update this dictionary as needed. For tools with multiple labels,
# use a comma-separated string, e.g., 'tool_name': 'label1,label2'.
reentrancy_labels = {
    'ccc': 'Reentrancy_Vulnerability',
    'confuzzius': 'Reentrancy',
    'conkas': 'Reentrancy', #.sol 0.5
    #'manticore-0.3.7': 'Reentrancy', # placeholder
    'mythril-0.24.7': 'State_access_after_external_call_SWC_107',
    'oyente+-2acaf2e': 'Re_Entrancy_Vulnerability',
    'securify': 'DAO', 
    #'securify2': 'Reentrancy', # does not work
    'sfuzz': 'Reentrancy', 
    'slither-0.11.3': 'reentrancy_eth,reentrancy_no_eth',
    #'smartcheck': 'Reentrancy', # never finds any occurrence of reentrancy
    'solhint-6.0.0': 'reentrancy',
    #'ethainter': 'Reentrancy', # does not work
    'ethor-2023': 'insecure',
    'oyente+-060ca34':'Callstack_Depth_Attack_Vulnerability',
    'vandal': 'ReentrantCall',
    'gpt-oss': 'reentrant',
    'gpt-5-mini': 'reentrant',
    'gpt-5': 'reentrant',
    'gpt-5-nano': 'reentrant'
    }

# 1. Determine the "true" reentrancy label for each file based on its filename.
# 'ree' followed by an optional number indicates a true reentrancy vulnerability.
# df['true_reentrancy'] = df['filename'].str.contains(r'ree\d*\.sol', case=False)
df['true_reentrancy'] = df['filename'].str.contains(r'_ree', case=False)

# 2. Determine the "predicted" reentrancy label based on the 'findings' column.
# This function will check if any of the tool-specific reentrancy labels are present in the findings.
def get_prediction(row):
    tool_id = row['toolid']
    findings = str(row['findings']) # Convert to string to handle potential NaN values
    
    # Check if the tool is in our labels dictionary.
    if tool_id in reentrancy_labels:
        # Split the tool's finding string into a list of individual labels.
        tool_findings = [f.strip() for f in reentrancy_labels[tool_id].split(',')]
        
        # Check if any of the tool's labels are present in the findings from the data.
        for label in tool_findings:
            if label in findings:
                return True
    return False

df['predicted_reentrancy'] = df.apply(get_prediction, axis=1)

# Save the DataFrame to a new CSV file.
df.to_csv('reentrancy_metrics_data.csv', index=False)

# 3. Calculate metrics for each unique tool and print only the results.
# Analyze only the tools present in the reentrancy_labels dictionary.
tools_to_analyze = reentrancy_labels.keys()

print("Reentrancy Metrics per Tool:")
print("=" * 30)

for tool in tools_to_analyze:
    
    # Filter the DataFrame for the current tool.
    tool_df = df[df['toolid'] == tool]
    
    # Calculate True Positives (TP), False Positives (FP), True Negatives (TN), and False Negatives (FN).
    TP = len(tool_df[(tool_df['true_reentrancy'] == True) & (tool_df['predicted_reentrancy'] == True)])
    FP = len(tool_df[(tool_df['true_reentrancy'] == False) & (tool_df['predicted_reentrancy'] == True)])
    TN = len(tool_df[(tool_df['true_reentrancy'] == False) & (tool_df['predicted_reentrancy'] == False)])
    FN = len(tool_df[(tool_df['true_reentrancy'] == True) & (tool_df['predicted_reentrancy'] == False)])
    
    # Calculate Accuracy, Precision, and Recall.
    # Handle cases where the denominator is zero to avoid errors.
    accuracy = (TP + TN) / (TP + FP + TN + FN) if (TP + FP + TN + FN) > 0 else 0
    
    # Precision: Out of all positive predictions, how many were correct?
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    
    # Recall: Out of all actual positives, how many were correctly predicted?
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    # Calculate the F1 Score
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    f1_score = 2 * TP / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0
    #f1_score = f1_score(tool_df['true_reentrancy'], tool_df['predicted_reentrancy'], zero_division=0, average = 'weighted')
    if f1_score > 0:
        print(f"Tool: {tool}")
        print(f"  Accuracy:  {accuracy:.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall:    {recall:.4f}")
        print(f"  F1 Score:  {f1_score:.4f}")
        print("-" * 30)


Reentrancy Metrics per Tool:
Tool: ccc
  Accuracy:  0.5714
  Precision: 0.7391
  Recall:    0.2208
  F1 Score:  0.3400
------------------------------
Tool: confuzzius
  Accuracy:  0.5909
  Precision: 0.5761
  Recall:    0.6883
  F1 Score:  0.6272
------------------------------
Tool: mythril-0.24.7
  Accuracy:  0.7013
  Precision: 0.6703
  Recall:    0.7922
  F1 Score:  0.7262
------------------------------
Tool: slither-0.11.3
  Accuracy:  0.7403
  Precision: 0.7033
  Recall:    0.8312
  F1 Score:  0.7619
------------------------------
Tool: solhint-6.0.0
  Accuracy:  0.5390
  Precision: 0.5882
  Recall:    0.2597
  F1 Score:  0.3604
------------------------------
Tool: oyente+-060ca34
  Accuracy:  0.5260
  Precision: 0.5200
  Recall:    0.6753
  F1 Score:  0.5876
------------------------------


In [6]:
import pandas as pd
from sklearn.metrics import f1_score

# Read data from the CSV file and select only the required columns.
df = pd.read_csv('results_aggregated_new_source.csv')
df = df[['filename', 'toolid', 'findings']]
print(df)

# Dictionary now stores a list of possible reentrancy labels per tool
reentrancy_labels = {
    'ccc': ['Reentrancy_Vulnerability'],
    'confuzzius': ['Reentrancy'],
    'conkas': ['Reentrancy'], 
    'mythril-0.24.7': ['State_access_after_external_call_SWC_107'],
    'oyente+-2acaf2e': ['Re_Entrancy_Vulnerability'],
    'securify': ['DAO'], 
    'sfuzz': ['Reentrancy'], 
    'slither-0.11.3': ['reentrancy_eth', 'reentrancy_no_eth'],
    'solhint-6.0.0': ['reentrancy'],
    'ethor-2023': ['insecure'],
    'oyente+-060ca34': ['Callstack_Depth_Attack_Vulnerability'],
    'vandal': ['ReentrantCall'],
    'gpt-oss': ['reentrant'],
    'gpt-5-mini': ['reentrant'],
    'gpt-5': ['reentrant'],
    'gpt-5-nano': ['reentrant']
}

# 1. True label from filename
df['true_reentrancy'] = df['filename'].str.contains(r'_ree', case=False)

# 2. Prediction function
def get_prediction(row):
    tool_id = row['toolid']
    findings = str(row['findings'])  # convert NaN to string

    if tool_id in reentrancy_labels:
        for label in reentrancy_labels[tool_id]:
            if label in findings:
                return True
    return False

def get_prediction(row):
    tool_id = row['toolid']
    findings = str(row['findings']).strip()

    if tool_id not in reentrancy_labels:
        return False

    # Remove surrounding braces if present and split on commas
    if findings.startswith("{") and findings.endswith("}"):
        findings_list = [f.strip() for f in findings[1:-1].split(",") if f.strip()]
    else:
        findings_list = [findings] if findings and findings != "nan" else []

    # Case-insensitive check
    for label in reentrancy_labels[tool_id]:
        if any(label.lower() == f.lower() for f in findings_list):
            return True
    return False


df['predicted_reentrancy'] = df.apply(get_prediction, axis=1)

# Save updated DataFrame
df.to_csv('reentrancy_metrics_data.csv', index=False)

# 3. Metrics per tool
tools_to_analyze = reentrancy_labels.keys()

print("Reentrancy Metrics per Tool:")
print("=" * 30)

for tool in tools_to_analyze:
    tool_df = df[df['toolid'] == tool]

    TP = len(tool_df[(tool_df['true_reentrancy'] == True) & (tool_df['predicted_reentrancy'] == True)])
    FP = len(tool_df[(tool_df['true_reentrancy'] == False) & (tool_df['predicted_reentrancy'] == True)])
    TN = len(tool_df[(tool_df['true_reentrancy'] == False) & (tool_df['predicted_reentrancy'] == False)])
    FN = len(tool_df[(tool_df['true_reentrancy'] == True) & (tool_df['predicted_reentrancy'] == False)])

    accuracy = (TP + TN) / (TP + FP + TN + FN) if (TP + FP + TN + FN) > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * TP / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0

    if f1 > 0:
        print(f"Tool: {tool}")
        print(f"  Accuracy:  {accuracy:.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall:    {recall:.4f}")
        print(f"  F1 Score:  {f1:.4f}")
        print("-" * 30)


                                               filename     toolid  \
0     tests/raw/safe/01dd4a2eacf1f751bcf9070bca20c3a...        ccc   
1     tests/raw/safe/04045504816e9b4137a6838c8a53dde...        ccc   
2     tests/raw/safe/083e60356eeecf6edf0e82b79f66a4b...        ccc   
3     tests/raw/safe/0a7dcd7ad58d91e0373b21e4e4522cd...        ccc   
4     tests/raw/reentrant/0cdc64278f3169e4b4be494f50...        ccc   
...                                                 ...        ...   
1288  tests/raw/safe/f21a824407dfb37c81f7fbbd8d8e0e5...  securify2   
1289  tests/raw/reentrant/f26ed098ddfa730dd9cebaaaab...  securify2   
1290  tests/raw/safe/f40afd3fefac7bcb1b05c424d48579a...  securify2   
1291  tests/raw/reentrant/f73768e16680cd9fffd24f529e...  securify2   
1292  tests/raw/reentrant/fc2b88d194347a48a9f16fd4ad...  securify2   

                                               findings  
0                                                    {}  
1                                          

In [4]:
import pandas as pd
from collections import defaultdict
from io import StringIO

def get_file_type(filename):
    """
    Extracts the file type from the path using a predefined dictionary.
    """
    d = {
        'tests/0_8/cross-contract/create/': 'create',
        'tests/0_8/cross-contract/gmx/': 'gmx',
        'tests/0_8/cross-contract/human/': 'human',
        'tests/0_8/cross-contract/read-only/': 'read-only',
        'tests/0_8/cross-contract/to-target/': 'cross-to-target',
        'tests/0_8/always-safe/underflow/': 'underflow',
        'tests/0_8/always-safe/emit/': 'emit',
        'tests/0_8/always-safe/constructor/': 'safe-constructor',
        'tests/0_8/always-safe/send-transfer/': 'send-transfer',
        'tests/0_8/always-safe/this/': 'safe-this',
        'tests/0_8/cross-function/guard/mutex/mod/': 'cross-function-mod',
        'tests/0_8/cross-function/guard/mutex/no-mod/': 'cross-function-mutex',
        'tests/0_8/single-function/low-level-call/to-sender/': 'LLC-to-sender',
        'tests/0_8/single-function/low-level-call/to-sender/guard/mutex/': 'LLC-to-sender-guard-mutex',
        'tests/0_8/single-function/low-level-call/to-sender/guard/access-control/': 'LLC-to-sender-guard-access-control',
        'tests/0_8/single-function/low-level-call/to-sender/guard/block-number/': 'LLC-to-sender-guard-block-number',
        'tests/0_8/single-function/low-level-call/to-sender/folded': 'LLC-to-sender-folded',
        'tests/0_8/single-function/low-level-call/to-sender/gas': 'LLC-to-sender-gas',
        'tests/0_8/single-function/low-level-call/to-target/': 'LLC-to-target',
        'tests/0_8/single-function/method-invocation/': 'single-cast'
    }
    cat = 'unknown'
    for prefix, category in d.items():
        if prefix in filename:
            cat = category

    return [cat]

# 3. Analyze and print TP, FP, TN, FN for each tool and category.
print("Detailed Reentrancy Analysis per Tool and Category:")
print("=" * 60)

for tool in reentrancy_labels.keys():
    print(f"Tool: {tool}")
    print("-" * 30)

    tool_df = df[df['toolid'] == tool].copy()
    
    # Store metrics for each category
    category_metrics = defaultdict(lambda: defaultdict(int))

    total_contracts = 0
    for _, row in tool_df.iterrows():
        categories = get_file_type(row['filename'])
        true_reentrancy = row['true_reentrancy']
        predicted_reentrancy = row['predicted_reentrancy']

        for category in categories:
            if true_reentrancy and predicted_reentrancy:
                category_metrics[category]['TP'] += 1
            elif not true_reentrancy and predicted_reentrancy:
                category_metrics[category]['FP'] += 1
            elif not true_reentrancy and not predicted_reentrancy:
                category_metrics[category]['TN'] += 1
            elif true_reentrancy and not predicted_reentrancy:
                category_metrics[category]['FN'] += 1
            
            category_metrics[category]['total'] += 1

    for category, metrics in sorted(category_metrics.items()):
        print(f"  Category: {category} (Total files: {metrics['total']})")
        print(f"    TP: {metrics['TP']}")
        print(f"    FP: {metrics['FP']}")
        print(f"    TN: {metrics['TN']}")
        print(f"    FN: {metrics['FN']}")
        print("-" * 30)
    print("=" * 60)
    total_contracts += sum(metrics['total'] for metrics in category_metrics.values())
    print('check if I counted correctly:', total_contracts)


Detailed Reentrancy Analysis per Tool and Category:
Tool: confuzzius
------------------------------
  Category: LLC-to-sender (Total files: 2)
    TP: 0
    FP: 0
    TN: 1
    FN: 1
------------------------------
  Category: LLC-to-sender-folded (Total files: 6)
    TP: 3
    FP: 1
    TN: 2
    FN: 0
------------------------------
  Category: LLC-to-sender-gas (Total files: 3)
    TP: 1
    FP: 0
    TN: 2
    FN: 0
------------------------------
  Category: LLC-to-sender-guard-access-control (Total files: 3)
    TP: 0
    FP: 0
    TN: 2
    FN: 1
------------------------------
  Category: LLC-to-sender-guard-block-number (Total files: 2)
    TP: 0
    FP: 0
    TN: 2
    FN: 0
------------------------------
  Category: LLC-to-sender-guard-mutex (Total files: 32)
    TP: 15
    FP: 7
    TN: 5
    FN: 5
------------------------------
  Category: LLC-to-target (Total files: 6)
    TP: 1
    FP: 1
    TN: 2
    FN: 2
------------------------------
  Category: create (Total files: 6)
 

In [ ]:
from collections import defaultdict
def get_file_type(filename):

    d = {
        'tests/0_8/cross-contract/create/': 'create',
        'tests/0_8/cross-contract/gmx/': 'gmx',
        'tests/0_8/cross-contract/human/': 'human',
        'tests/0_8/cross-contract/read-only/': 'read-only',
        'tests/0_8/cross-contract/to-target/': 'cross-to-target',
        'tests/0_8/always-safe/underflow/': 'underflow',
        'tests/0_8/always-safe/emit/': 'emit',
        'tests/0_8/always-safe/constructor/': 'safe-constructor',
        'tests/0_8/always-safe/send-transfer/': 'send-transfer',
        'tests/0_8/always-safe/this/': 'safe-this',
        'tests/0_8/cross-function/guard/mutex/mod/': 'cross-function-mod',
        'tests/0_8/cross-function/guard/mutex/no-mod/': 'cross-function-mutex',
        'tests/0_8/single-function/low-level-call/to-sender/': 'LLC-to-sender',
        'tests/0_8/single-function/low-level-call/to-sender/guard': 'LLC-to-sender-guard',
        'tests/0_8/single-function/low-level-call/to-sender/folded': 'LLC-to-sender-folded',
        'tests/0_8/single-function/low-level-call/to-sender/gas': 'LLC-to-sender-gas',
        'tests/0_8/single-function/low-level-call/to-target/': 'LLC-to-target',
        'tests/0_8/single-function/method-invocation/': 'single-cast'
    }
    cat = 'unknown'
    for prefix, category in d.items():
        if prefix in filename:
            cat = category

    return [cat]


for tool in tools_to_analyze:
    tool_df = df[df['toolid'] == tool]

    # Analyze False Positives (FP)
    false_positives = tool_df[(tool_df['true_reentrancy'] == False) & (tool_df['predicted_reentrancy'] == True)]
    fp_categories = defaultdict(int)
    for _, row in false_positives.iterrows():
        categories = get_file_type(row['filename'])
        for cat in categories:
            fp_categories[cat] += 1
    
    # Analyze False Negatives (FN)
    false_negatives = tool_df[(tool_df['true_reentrancy'] == True) & (tool_df['predicted_reentrancy'] == False)]
    fn_categories = defaultdict(int)
    for _, row in false_negatives.iterrows():
        categories = get_file_type(row['filename'])
        for cat in categories:
            fn_categories[cat] += 1

    print(f"Tool: {tool}")
    print("  False Positives:")
    if fp_categories:
        for cat, count in sorted(fp_categories.items()):
            print(f"    - {cat}: {count}")
    else:
        print("    No false positives found.")
    
    print("  False Negatives:")
    if fn_categories:
        for cat, count in sorted(fn_categories.items()):
            print(f"    - {cat}: {count}")
    else:
        print("    No false negatives found.")
    print("-" * 30)


Tool: confuzzius
  False Positives:
    No false positives found.
  False Negatives:
    No false negatives found.
------------------------------
Tool: mythril-0.24.7
  False Positives:
    No false positives found.
  False Negatives:
    No false negatives found.
------------------------------
Tool: oyente+-2acaf2e
  False Positives:
    No false positives found.
  False Negatives:
    No false negatives found.
------------------------------
Tool: slither-0.11.3
  False Positives:
    No false positives found.
  False Negatives:
    No false negatives found.
------------------------------
Tool: solhint-6.0.0
  False Positives:
    No false positives found.
  False Negatives:
    No false negatives found.
------------------------------
Tool: oyente+-060ca34
  False Positives:
    No false positives found.
  False Negatives:
    No false negatives found.
------------------------------
Tool: vandal
  False Positives:
    No false positives found.
  False Negatives:
    No false negatives